# Modelo Perceptrón Multicapa
[MLP - Sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys

In [5]:
# Añadimos la carpeta 'drive' al path
ruta_carpeta_drive = os.path.abspath('../../drive')
if ruta_carpeta_drive not in sys.path:
    sys.path.insert(0, ruta_carpeta_drive)

import drive

Para entrenar este modelo hemos partido del conjunto de datos ya limpio con la variable GenresList transformada con OneHotEncoding.

In [6]:
# Descargamos los datos en formato parquet de Google Drive
drive.descargar_archivos_concretos('datosEntrenamiento.parquet', '../../drive')

Archivo datosEntrenamiento.parquet guardado en: ../data/clean\datosEntrenamiento.parquet


In [7]:
data = pd.read_parquet('../data/clean/datosEntrenamiento.parquet')

In [8]:
data.head()

,Bestseller,NumPages,SagaNumber,RedPerc,BluePerc,BelongsSaga,Price,WordsTitle,PriceFormat,BookInterest1M,...,World History,World War I,World War II,Writing,Young Adult,Young Adult Contemporary,Young Adult Fantasy,Young Adult Romance,Young Adult Science Fiction,Zombies
0,0.0,329.0,1.0,0.51,0.40,0,19.99,1.0,paperback,0.0,...,0,0,0,0,0,0,0,0,0,0
1,0.0,269.0,2.0,0.61,0.54,1,3.99,2.0,ebook,0.0,...,0,0,0,0,0,0,0,0,0,0
2,0.0,2335.0,1.0,0.72,0.57,1,20.99,7.0,ebook,0.0,...,0,0,0,0,1,0,0,0,0,0
3,0.0,40.0,1.0,0.83,0.35,0,25.00,1.0,hardcover,0.0,...,0,0,0,0,1,0,0,0,0,0
4,0.0,189.0,1.0,0.59,0.26,0,15.00,4.0,paperback,0.0,...,0,0,0,0,0,0,0,0,0,0


## Ajuste de hiperparámetros

In [63]:
import os
import numpy as np

from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, KFold, cross_validate
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import balanced_accuracy_score

In [64]:
# Semilla
SEED = 22

# Proporción del conjunto de test
TEST_SIZE = 0.3

# Número de folds para la validación cruzada
CV_FOLDS = 5

In [65]:
def codificarPriceFormat(df):
    return pd.get_dummies(df, columns=['PriceFormat'], dtype=int)

### Métricas de evaluación

In [66]:
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix

In [67]:
# Función para calcular la sensibilidad
def sensitivity(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tp / (tp + fn)

# Función para calcular la especificidad
def specificity(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn + fp)

# Convertir las funciones en funciones de puntuación
sensitivity_scorer = make_scorer(sensitivity)
specificity_scorer = make_scorer(specificity)

METRICS = {'balanced_accuracy': 'balanced_accuracy',
           'sensitivity': sensitivity_scorer,
           'specificity': specificity_scorer}

### Preparamos entorno de Mlflow

In [68]:
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature

In [69]:
# Sets the sqlite db as the MLFLOW_TRACKING_URI 
os.environ['MLFLOW_TRACKING_URI'] = 'sqlite:///mlruns.db'

# WARNING: TO SEE THE LOCAL SERVER YOU HAVE TO CHOOSE THE CORRECT BACKEND STORE AS FOLLOWS:
# mlflow ui --port 8080 --backend-store-uri sqlite:///mlruns.db


# mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

# Para imprimir los experimentos que están en la base de datos

# Establecer la URI de seguimiento
mlflow.set_tracking_uri('sqlite:///mlruns.db')

# Obtener todos los experimentos
experiment_ids = mlflow.search_runs().experiment_id.unique()

# Imprimir los experimentos
for exp_id in experiment_ids:
    print(exp_id)
    
# Defino el experimento el que guardaré todas las ejecuciones
mlflow.set_experiment(experiment_name = 'MultiLayerPerceptron')

2


<Experiment: artifact_location='/Users/maria/Dropbox/UCM/PD1/pruebas-proyecto/mlruns/2', creation_time=1713190833047, experiment_id='2', last_update_time=1713190833047, lifecycle_stage='active', name='MultiLayerPerceptron', tags={}>

### Baseline Model

Creamos un modelo base para después comparar con los otros modelos que entrenemos con más técnicas de procesado y transformaciones. 

Modelo baseline:
* Parámetros por defecto
* Todas las variables
* Sin transformaciones

In [70]:
from sklearn.neural_network import MLPClassifier
import sklearn

In [73]:
dataBaseline = data.copy()
dataBaseline = codificarPriceFormat(dataBaseline)

In [74]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_base = dataBaseline.drop('Bestseller', axis=1)
y_base = dataBaseline['Bestseller']
X_base_train, X_base_test, y_base_train, y_base_test = train_test_split(X_base, y_base, test_size=TEST_SIZE, stratify=y, random_state=SEED)

In [75]:
PMC = sklearn.neural_network.MLPClassifier(random_state=SEED)

In [76]:
scores = cross_validate(PMC, X_base_train, y_base_train, scoring=METRICS, cv=CV_FOLDS,
                        return_train_score=True, verbose=1, n_jobs=-1)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    6.0s remaining:    9.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    6.7s finished


In [77]:
scores

{'fit_time': array([4.52730989, 4.98621488, 4.30443001, 5.00402904, 3.45042109]),
 'score_time': array([0.00933719, 0.00609207, 0.0111661 , 0.00590992, 0.01234126]),
 'test_balanced_accuracy': array([0.69871795, 0.73477017, 0.68433503, 0.67843344, 0.72431441]),
 'train_balanced_accuracy': array([0.75524042, 0.78301958, 0.81623199, 0.84175615, 0.81962251]),
 'test_sensitivity': array([0.3974359 , 0.51282051, 0.38461538, 0.42307692, 0.48051948]),
 'train_sensitivity': array([0.51446945, 0.58199357, 0.63987138, 0.6977492 , 0.67628205]),
 'test_specificity': array([1.        , 0.95671982, 0.98405467, 0.93378995, 0.96810934]),
 'train_specificity': array([0.9960114 , 0.98404558, 0.99259259, 0.9857631 , 0.96296296])}

In [19]:
# Registramos los resultados en MlFlow
with mlflow.start_run():
    
    for metric in METRICS.keys():
        # Calculate the mean values for train and test sets
        train_mean = np.mean(scores[f"train_{metric}"])
        test_mean = np.mean(scores[f"test_{metric}"])

        # Log the mean values for train and test sets
        mlflow.log_metric(f"train_{metric}_mean", train_mean)
        mlflow.log_metric(f"test_{metric}_mean", test_mean)

    # Establece una etiqueta que describe el propósito de esta ejecución
    mlflow.set_tag("Información", "Baseline (default)")

    # Infiere el signature del modelo, que describe el tipo de entrada y salida del modelo
    PMC.fit(X_base_train, y_base_train)
    signature = infer_signature(X_base_train, PMC.predict(X_base_train))

    # Registra el modelo
    model_info = mlflow.sklearn.log_model(
        sk_model=PMC,
        artifact_path="mlp_model",
        signature=signature,
        input_example=X_base_train,
        registered_model_name="baseline",
    )

2024/04/15 23:18:00 WARNING mlflow.utils.validation: Tag value '[{"run_id": "45d9c23c76b54c5b9e5f981eb9b4f175", "artifact_path": "mlp_model", "utc_time_created": "2...' (24513 characters) is truncated to 5000 characters to meet the length limit.
Registered model 'baseline' already exists. Creating a new version of this model...
Created version '3' of model 'baseline'.


### Escalado de variables

Para mejorar el rendimiento de la red neuronal, vamos a escalar las variables. Como los datos tienen muchos outliers, una estandarización sería muy susceptible a valores extremos. Por lo tanto, vamos a aplicar un [escalado robusto](https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html#plot-all-scaling-robust-scaler-section), que usa la mediana y los rangos intercuartílicos. De esta forma la transformación será más resistente a las variaciones introducidas por datos atípicos.

In [78]:
from sklearn.preprocessing import RobustScaler

In [79]:
data_scaled = data.copy()
X_scaled = data_scaled.drop('Bestseller', axis=1)
y_scaled = data_scaled['Bestseller']

# Dividimos en train y test
X_scaled_train, X_scaled_test, y_scaled_train, y_scaled_test = train_test_split(X_scaled, y_scaled, test_size=TEST_SIZE, stratify=y, random_state=SEED)

In [80]:
# Inicializamos RobustScaler
scaler = RobustScaler()

# Solo lo aplicamos a las variables numéricas
# Incluimos 'Zombies' que no hay que escalar porque el RobustScaler ignora la primera columna de la lista 
# (hemos hecho pruebas)
variables_numericas = ['Zombies', 'SagaNumber', 'NumPages', 'RedPerc', 'BluePerc', 'Price', 'WordsTitle', 'BookInterest1M',
                     'Rating20Days', 'PrevBestSellAuthor']

# Aplicamos el RobustScaler a los datos de entrenamiento y test
X_scaled_train[variables_numericas] = scaler.fit_transform(X_scaled_train[variables_numericas])
X_scaled_test[variables_numericas] = scaler.transform(X_scaled_test[variables_numericas])

### Transformación de variables para el entrenamiento

Vamos a transformar la columna 'PriceFormat' con variables dummy para poder entrenar los modelos después.

In [81]:
X_to_train = X_scaled_train.copy()
X_to_train = codificarPriceFormat(X_to_train)

X_to_test = X_scaled_test.copy()
X_to_test = codificarPriceFormat(X_to_test)

### Creación de KFolds

Estrategia de [validación cruzada](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html) con k folds

In [82]:
from sklearn.model_selection import StratifiedKFold

In [83]:
# Inicializamos el objeto KFold
kf = StratifiedKFold(n_splits=CV_FOLDS, shuffle=True, random_state=SEED)

### Creación del pipeline

Creamos un pipeline con las operaciones que se deben aplicar a cada fold en el entrenamiento:
* Oversampling (SMOTENC)
* Redondear variables enteras
* Transformación variables categóricas con un valor único
* Clasificador (MLP)

In [84]:
from imblearn.over_sampling import SMOTENC
from imblearn.pipeline import Pipeline

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer

In [85]:
def redondearVariables(X):
    variablesRedondeo = ["NumPages", "SagaNumber", "WordsTitle"]
    # Itera sobre las columnas especificadas y redondea sus valores
    for v in variablesRedondeo:
        X[v] = np.round(X[v])
    return X

In [86]:
# Columnas de los géneros
columnas_generos = X_scaled_train.columns[14:]

# Columnas categóricas
categoricalColumns = ["BelongsSaga", "PriceFormat", 'HasTwitter', 'HasWikipedia'] + list(columnas_generos)

In [87]:
smote = SMOTENC(categorical_features = categoricalColumns, random_state = SEED)

# Definimos el clasificador 
PMC = sklearn.neural_network.MLPClassifier(random_state=SEED)

# Definimos el transformador para codificar la variable categórica 'PriceFormat'
column_transformer = ColumnTransformer([
    ('ohe', OneHotEncoder(), ['PriceFormat'])
], remainder='passthrough')

# Definimos el transformador de la función para redondear
transformador_funcion = FunctionTransformer(func=redondearVariables)

# Construimos el pipeline
pipeline = Pipeline([
    ('smote', smote),
    ('redondear_variables', transformador_funcion),
    ('encoder', column_transformer),
    ('classifier', PMC)
])

### Búsqueda de hiperparámetros

Vamos a buscar los hiperparámetros que optimicen el modelo probando una serie de combinaciones. Para ello, vamos a usar:
* Datos con escalado robusto
* Pipeline previamente creado
* Cross validation con k folds

Como las redes neuronales son muy costosas computacionalmente, vamos a dividir el ajuste de hiperparámetros en tres pasos.  Los dos primeros reducen mucho el número de opciones y por lo tanto optimizan el tiempo de ejecución.

1. Elegir la arquitectura más óptima de una serie de opciones mediante cross-validation con k folds
2. Ajustar el resto de hiperparámetros mediante GridSearch y RandomSearch tras fijar dicha arquitectura (para reducir el tiempo de entrenamiento)
3. Ajustar todos los hiperparámetros a la vez (incluida la arquitectura) mediante GridSearch y RandomSearch

Además, vamos a fijar la función de activación a 'logistic', que es la indicada para cuando la salida es una probabilidad.

### 1. Ajuste de la arquitectura de la red neuronal

In [88]:
# Definimos varias opciones para la distribución de las capas de neuronas
layers = [(50,), (100,), (50, 50), (100, 100), (200,), (150, 150), (200, 100), (100, 50, 25), (150, 100, 50)]

# Inicializamos la lista de scores
scores = {}

# Iterate over the list of hidden layer sizes
for hidden_layer_sizes in layers:
    # Set parameters for the neural network model inside the pipeline
    params = {'classifier__hidden_layer_sizes': hidden_layer_sizes,
              'classifier__activation':'logistic'}
    pipeline.set_params(**params)
    
    # Perform cross-validation
    cv_results = cross_validate(estimator=pipeline, X=X_scaled_train, y=y_scaled_train,
                                scoring=METRICS, cv=kf, return_train_score=True,
                                verbose=1, n_jobs=-1)
    
    # Append the results to the scores list
    scores[hidden_layer_sizes] = cv_results

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
/Users/maria/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/maria/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/maria/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   18.8s remaining:   28.2s
/Users/maria/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWar

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
/Users/maria/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/maria/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   52.2s remaining:  1.3min
/Users/maria/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/maria/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWar

In [89]:
scores

{(50,): {'fit_time': array([16.83218098, 16.72554803, 16.74496317, 16.72596002, 16.78933907]),
  'score_time': array([0.01013303, 0.01168394, 0.01319385, 0.01171875, 0.011307  ]),
  'test_balanced_accuracy': array([0.76213422, 0.75715496, 0.74902167, 0.73068142, 0.73366269]),
  'train_balanced_accuracy': array([0.84260771, 0.83540367, 0.83060067, 0.844902  , 0.83696581]),
  'test_sensitivity': array([0.61538462, 0.62820513, 0.56410256, 0.56410256, 0.55844156]),
  'train_sensitivity': array([0.74276527, 0.73633441, 0.7170418 , 0.74276527, 0.73717949]),
  'test_specificity': array([0.90888383, 0.88610478, 0.93394077, 0.89726027, 0.90888383]),
  'train_specificity': array([0.94245014, 0.93447293, 0.94415954, 0.94703872, 0.93675214])},
 (100,): {'fit_time': array([21.79483604, 22.00852871, 22.54661322, 22.43860912, 22.36440492]),
  'score_time': array([0.00732088, 0.00870895, 0.01000285, 0.01294088, 0.01279521]),
  'test_balanced_accuracy': array([0.76441213, 0.7447579 , 0.7350622 , 0.7185

Estudiamos las gráficas de las métricas para cada distribución de las capas para elegir la óptima y poder ajustar el resto de hiperparámetros.

In [1]:
# Initialize subplots
fig, axes = plt.subplots(3, 1, figsize=(10, 15))

for i, metric in enumerate(METRICS.keys()):
    # Initialize lists to store mean scores
    train_means = []
    test_means = []
    
    # Recorremos las capas
    for layer in layers:
        # Calculamos la media de esa métrica de entrenamiento y test
        train_mean = np.mean(scores[layer][f"train_{metric}"])
        test_mean = np.mean(scores[layer][f"test_{metric}"])

        # Append to lists
        train_means.append(train_mean)
        test_means.append(test_mean)
    
    # Mostramos las métricas en la gráfica
    axes[i].bar(range(len(layers)), train_means, label='Train')
    axes[i].bar(range(len(layers)), test_means, label='Test')
    
    # Set labels and title
    axes[i].set_xlabel('Layers')
    axes[i].set_ylabel(metric)
    axes[i].set_title(f'Mean {metric} for Different Layer Configurations')
    
    # Set xticks
    axes[i].set_xticks(range(len(layers)))
    axes[i].set_xticklabels(layers)
        
    # Add legend
    axes[i].legend()

# Adjust layout
plt.tight_layout()

# Show plots
plt.show()

NameError: name 'plt' is not defined

Estudiando las gráficas, observamos que tanto para balanced_accuracy como para sensitivity el valor de la métrica en el test encuentra su máximo en una distribución de (200, 100).
Aunque no es el caso para specificity (que se maximiza en (100, 100)), consigue un valor muy alto de 0.9.

Por lo tanto, elegimos la estructura (200, 100) apoyándonos en las dos primeras métricas para continuar ajustando los hiperparámetros en el siguiente paso.

In [91]:
LAYERS = (200, 100)

In [92]:
# Registramos los resultados en MlFlow del modelo con la mejor distribución de capas de neuronas
best_scores = scores[LAYERS]
best_params = {'classifier__hidden_layer_sizes': LAYERS,
              'classifier__activation':'logistic'}

# Como hemos usado cross validation esta vez, tenemos que crear un nuevo modelo con los parametros deseados
best_model = MLPClassifier(hidden_layer_sizes=LAYERS,
                           activation='logistic')


with mlflow.start_run():
    
    # Calculamos la media de las métricas del mejor parámetro y las almacenamos
    for metric in METRICS.keys():
        # Calculate the mean values for train and test sets
        train_mean = np.mean(best_scores[f"train_{metric}"])
        test_mean = np.mean(best_scores[f"test_{metric}"])
        
        # Calculate the standard deviations for train and test sets
        train_std = np.std(best_scores[f"train_{metric}"])
        test_std = np.std(best_scores[f"test_{metric}"])

        # Log the standard deviations for train and test sets
        mlflow.log_metric(f"train_{metric}_std", train_std)
        mlflow.log_metric(f"test_{metric}_std", test_std)

        # Log the mean values for train and test sets
        mlflow.log_metric(f"train_{metric}_mean", train_mean)
        mlflow.log_metric(f"test_{metric}_mean", test_mean)
    
    # Almacenamos los valores de los hiperparámetros
    for key, value in best_params.items():
        mlflow.log_param(key, value)
    
    # Establecemos una etiqueta que describe el propósito de esta ejecución
    mlflow.set_tag("Información", "Ajuste capas neuronas. Con dataset escalado")

    # Inferimos el signature del modelo, que describe el tipo de entrada y salida del modelo
    best_model.fit(X_to_train, y_scaled_train) # en este caso le metemos el df con las dummies de PriceFormat porque el modelo no tiene pipeline porque es el básico
    signature = infer_signature(X_to_train, best_model.predict(X_to_train))

    # Registramos el modelo
    model_info = mlflow.sklearn.log_model(
        sk_model=best_model,
        artifact_path="mlp_model",
        signature=signature,
        input_example=X_to_train,
        registered_model_name="ajuste_capas",
    )

2024/04/16 01:00:54 WARNING mlflow.utils.validation: Tag value '[{"run_id": "4fd3bf04dd0e4598852f363f0a3e40fb", "artifact_path": "mlp_model", "utc_time_created": "2...' (24515 characters) is truncated to 5000 characters to meet the length limit.
Registered model 'ajuste_capas' already exists. Creating a new version of this model...
Created version '3' of model 'ajuste_capas'.


### 2. Ajuste hiperparámetros adicionales

Vamos a determinar algunos parámetros por defecto para mejorar la eficiencia del modelo:
* **hidden_layer_sizes:** (200, 100) - obtenidos del experimento anterior
* **activation:** 'logistic' - la salida es una probabilidad
* **early_stopping:** True - detiene el entrenamiento de modelos cuando métricas que no mejoran
* **solver:** 'adam' - método por defecto. Optimiza el descenso de gradiente estocástico y se recomienda para datasets grandes (más de mil filas de entrenamiento)

### 2.1 Grid Search
[GridSearchCV - Sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)

In [114]:
# Definir los hiperparámetros a ajustar
param_grid = {
    'classifier__hidden_layer_sizes': [LAYERS],
    'classifier__activation': ['logistic'],
    'classifier__early_stopping': [True],
    'classifier__alpha': [0.00001, 0.0001, 0.001, 0.01, 0.1, 0.5, 1],
    'classifier__learning_rate': ['constant', 'invscaling', 'adaptive'],
    'classifier__learning_rate_init': [0.0001, 0.001, 0.01, 0.1]
}

# Inicializo GridSearch
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=kf,
                           scoring=METRICS, return_train_score=True,
                           refit='balanced_accuracy',
                           verbose=1, n_jobs=-1)

grid_search.fit(X_scaled_train, y_scaled_train)

# Resultados
cv_results_gr = grid_search.cv_results_
best_params_gr = grid_search.best_params_

Fitting 5 folds for each of 60 candidates, totalling 300 fits


In [97]:
cv_results_gr

{'mean_fit_time': array([19.11752272, 18.77449269, 24.40174956, 20.71089773, 18.98369942,
        24.21967726, 21.47990651, 20.5531374 , 24.73145857, 21.33887901,
        22.04839377, 23.213095  , 20.67812881, 23.62861156, 23.90043459,
        21.19666862, 21.28677654, 23.56978478, 21.59832582, 20.63191767,
        21.66762209, 20.99792418, 20.89926395, 22.73365321, 21.53834262,
        19.37656813, 20.27246184, 19.83541069, 20.66381187, 21.17093925,
        19.99390898, 20.56205535, 20.36872659, 20.45214963, 21.63406482,
        18.009623  ]),
 'std_fit_time': array([0.16449517, 3.2968726 , 1.24654138, 0.6970222 , 2.45150577,
        1.80925849, 0.97201096, 2.58884194, 1.24665277, 0.64091649,
        2.77594858, 2.47037745, 1.29256375, 2.35125206, 1.85685392,
        0.40904437, 2.97220781, 2.99442069, 1.16715374, 2.3205102 ,
        2.77938604, 1.2130235 , 2.63026699, 2.09328037, 1.00059098,
        2.06125787, 1.9136437 , 1.68672614, 3.29187436, 1.72820518,
        1.34659078, 2.723

In [116]:
best_params_gr

{'classifier__activation': 'logistic',
 'classifier__alpha': 0.1,
 'classifier__early_stopping': True,
 'classifier__hidden_layer_sizes': (200, 100),
 'classifier__learning_rate': 'constant',
 'classifier__learning_rate_init': 0.1}

In [117]:
# Convertimos los resultados de la validación cruzada en un dataframe
df_results = pd.DataFrame(grid_search.cv_results_)
df_results

# Filtrando la fila con condiciones específicas
filtered_row = df_results.loc[
    (df_results['param_classifier__activation'] == 'logistic') &
    (df_results['param_classifier__alpha'] == 0.1) &
    (df_results['param_classifier__early_stopping'] == True) &
    (df_results['param_classifier__hidden_layer_sizes'] == (200, 100)) &
    (df_results['param_classifier__learning_rate'] == 'constant') &
    (df_results['param_classifier__learning_rate_init'] == 0.1)
]

index_row = filtered_row.index[0]

# Registro los resultados en MLFlow
with mlflow.start_run():

    # Almaceno los valores de los hiperparámetros
    for key, value in best_params.items():
        mlflow.log_param(key, value)

    # Registra las métricas de cada fold para cada métrica
    for metric in METRICS.keys():
        
        # Media
        mlflow.log_metric(f"mean_train_{metric}", df_results[f"mean_train_{metric}"][index_row])
        mlflow.log_metric(f"mean_test_{metric}", df_results[f"mean_test_{metric}"][index_row])

        # Desviación típica
        mlflow.log_metric(f"std_train_{metric}", df_results[f"std_train_{metric}"][index_row])
        mlflow.log_metric(f"std_test_{metric}", df_results[f"std_test_{metric}"][index_row])

        for i in range(5):

            # Resultados de entrenamiento en cada fold
            mlflow.log_metric(f"train_{metric}fold{i}", df_results[f"split{i}_train_{metric}"][index_row])
            # Resultados de validación en cada fold
            mlflow.log_metric(f"test_{metric}fold{i}", df_results[f"split{i}_test_{metric}"][index_row])

    # Establece una etiqueta que describe el propósito de esta ejecución
    mlflow.set_tag("Información", "GridSearch 60 combinaciones")

    # Infiere el signature del modelo, que describe el tipo de entrada y salida del modelo
    signature = infer_signature(X_scaled_train, grid_search.best_estimator_.predict(X_scaled_train))

    # Registra el modelo
    model_info = mlflow.sklearn.log_model(
        sk_model=grid_search,
        artifact_path="mlp_model",
        signature=signature,
        input_example=X_scaled_train,
        registered_model_name="mlp_gridsearch",
    )

2024/04/16 12:57:42 WARNING mlflow.utils.validation: Tag value '[{"run_id": "7451e0931e3b488ab0bc1c47d52a6036", "artifact_path": "mlp_model", "utc_time_created": "2...' (24353 characters) is truncated to 5000 characters to meet the length limit.
Registered model 'mlp_gridsearch' already exists. Creating a new version of this model...
Created version '2' of model 'mlp_gridsearch'.


### 2.2 Random Search

[RandomizedSearchCV - sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)

Con esta estrategia podemos aprovechar un rango más amplio de posibles hiperparámetros sin comprometer la capacidad computacional, ya que coge un número fijo (`n_iter`) de combinaciones aleatorias dentro de los rangos establecidos. 

Para beneficiarnos más de esta técnica y aumentar la variedad, vamos a presentar los posibles valores de los parámetros continuos como distribuciones.

In [104]:
from scipy.stats import uniform, randint

In [115]:
# Defino las distribuciones de los parámetros
param_dist = {
    'classifier__hidden_layer_sizes': [LAYERS],
    'classifier__activation': ['logistic'],
    'classifier__early_stopping': [True],
    'classifier__alpha': uniform(0.00001, 1),
    'classifier__learning_rate': ['constant', 'invscaling', 'adaptive'],
    'classifier__learning_rate_init': uniform(0.0001, 0.1)
}

# Inicializo RandomizedSearch
N_ITER = 84
random_search = RandomizedSearchCV(estimator=pipeline, param_distributions=param_dist, n_iter=N_ITER, cv=kf,
                                    scoring=METRICS, return_train_score=True,
                                    refit="balanced_accuracy",
                                    verbose=1, n_jobs=-1)

random_search.fit(X_scaled_train, y_scaled_train)

# Resultados
cv_results_rand = random_search.cv_results_
best_params_rand = random_search.best_params_

Fitting 5 folds for each of 60 candidates, totalling 300 fits


In [106]:
cv_results_rand

{'mean_fit_time': array([10.43667202, 10.51972313, 10.2704793 , 10.54374394, 10.17414355,
        11.75921111, 11.29780021, 10.52749019, 10.65777373, 10.13449726,
        10.89603715,  9.96843443,  9.04172726, 11.28343611, 10.01293731,
        10.07871742, 10.57651658,  9.97921057, 10.60975146, 10.18692541,
        10.2400939 , 11.18180799, 11.40704703, 10.86707549, 10.75598526,
        11.13230743, 11.33499613, 12.22195711, 11.3056076 , 11.69550619,
        12.87504063, 10.68090658, 11.88500037, 10.66409669, 12.21386223,
         9.25661139]),
 'std_fit_time': array([0.77723931, 1.3076297 , 1.21120376, 1.06077212, 0.56995398,
        0.73383813, 1.15024295, 1.14801333, 0.45434988, 0.74715547,
        0.53616361, 0.52649417, 0.10949126, 0.30305074, 0.93709234,
        0.8307621 , 1.30097406, 0.49311897, 0.53919215, 0.93999816,
        0.55349748, 0.51086712, 0.92480903, 1.01830951, 0.86146062,
        0.88443561, 0.7031484 , 1.25872576, 0.59214285, 1.23083131,
        1.34021721, 0.632

In [118]:
best_params_rand

{'classifier__activation': 'logistic',
 'classifier__alpha': 0.0887386581481309,
 'classifier__early_stopping': True,
 'classifier__hidden_layer_sizes': (200, 100),
 'classifier__learning_rate': 'adaptive',
 'classifier__learning_rate_init': 0.09309893159968192}

In [119]:
# Convertimos los resultados de la validación cruzada en un dataframe
df_results = pd.DataFrame(random_search.cv_results_)
df_results

# Filtrando la fila con condiciones específicas
filtered_row = df_results.loc[
    (df_results['param_classifier__activation'] == 'logistic') &
    (df_results['param_classifier__alpha'] == 0.0887386581481309) &
    (df_results['param_classifier__early_stopping'] == True) &
    (df_results['param_classifier__hidden_layer_sizes'] == (200, 100)) &
    (df_results['param_classifier__learning_rate'] == 'adaptive') &
    (df_results['param_classifier__learning_rate_init'] == 0.09309893159968192)
]

index_row = filtered_row.index[0]

# Registro los resultados en MLFlow
with mlflow.start_run():

    # Almaceno los valores de los hiperparámetros
    for key, value in best_params.items():
        mlflow.log_param(key, value)

    # Registra las métricas de cada fold para cada métrica
    for metric in METRICS.keys():
        
        # Media
        mlflow.log_metric(f"mean_train_{metric}", df_results[f"mean_train_{metric}"][index_row])
        mlflow.log_metric(f"mean_test_{metric}", df_results[f"mean_test_{metric}"][index_row])

        # Desviación típica
        mlflow.log_metric(f"std_train_{metric}", df_results[f"std_train_{metric}"][index_row])
        mlflow.log_metric(f"std_test_{metric}", df_results[f"std_test_{metric}"][index_row])

        for i in range(5):

            # Resultados de entrenamiento en cada fold
            mlflow.log_metric(f"train_{metric}fold{i}", df_results[f"split{i}_train_{metric}"][index_row])
            # Resultados de validación en cada fold
            mlflow.log_metric(f"test_{metric}fold{i}", df_results[f"split{i}_test_{metric}"][index_row])

    # Establece una etiqueta que describe el propósito de esta ejecución
    mlflow.set_tag("Información", "RandomizedSearch 60 combinaciones")

    # Infiere el signature del modelo, que describe el tipo de entrada y salida del modelo
    signature = infer_signature(X_scaled_train, random_search.best_estimator_.predict(X_scaled_train))

    # Registra el modelo
    model_info = mlflow.sklearn.log_model(
        sk_model=random_search,
        artifact_path="mlp_model",
        signature=signature,
        input_example=X_scaled_train,
        registered_model_name="mlp_randomizedsearch",
    )

2024/04/16 12:58:27 WARNING mlflow.utils.validation: Tag value '[{"run_id": "a6e5a4cdffc94b2097820ba4e545f13d", "artifact_path": "mlp_model", "utc_time_created": "2...' (24353 characters) is truncated to 5000 characters to meet the length limit.
Registered model 'mlp_randomizedsearch' already exists. Creating a new version of this model...
Created version '3' of model 'mlp_randomizedsearch'.


### 3. Ajuste combinado de hiperparámetros
Vamos a intentar aplicar las estrategias de búsqueda a todos los hiperparámetros a la vez para ver si mejora el resultado.

### 3.1 Grid Search
[GridSearchCV - Sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)

In [125]:
# Definir los hiperparámetros a ajustar (144)
# Dejamos solo las opciones de arquitectura que eran las más prometedoras en la prueba individual (2 capas)
param_grid = {
    'classifier__hidden_layer_sizes': [(50, 50), (100, 100), (150, 150), (200, 100)],
    'classifier__activation': ['logistic'],
    'classifier__early_stopping': [True],
    'classifier__alpha': [0.0001, 0.001, 0.01, 0.1, 0.5, 1],
    'classifier__learning_rate': ['constant', 'invscaling', 'adaptive'],
    'classifier__learning_rate_init': [0.001, 0.01, 0.1]
}

# Inicializo GridSearch
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=kf,
                           scoring=METRICS, return_train_score=True,
                           refit='balanced_accuracy',
                           verbose=1, n_jobs=-1)

grid_search.fit(X_scaled_train, y_scaled_train)

# Resultados
cv_results_gr = grid_search.cv_results_
best_params_gr = grid_search.best_params_

Fitting 5 folds for each of 144 candidates, totalling 720 fits


In [122]:
best_params_gr

{'classifier__activation': 'logistic',
 'classifier__alpha': 0.1,
 'classifier__early_stopping': True,
 'classifier__hidden_layer_sizes': (100, 100),
 'classifier__learning_rate': 'constant',
 'classifier__learning_rate_init': 0.1}

In [128]:
# Convertimos los resultados de la validación cruzada en un dataframe
df_results = pd.DataFrame(grid_search.cv_results_)
df_results

# Filtrando la fila con condiciones específicas
filtered_row = df_results.loc[
    (df_results['param_classifier__activation'] == 'logistic') &
    (df_results['param_classifier__alpha'] == 0.1) &
    (df_results['param_classifier__early_stopping'] == True) &
    (df_results['param_classifier__hidden_layer_sizes'] == (100, 100)) &
    (df_results['param_classifier__learning_rate'] == 'constant') &
    (df_results['param_classifier__learning_rate_init'] == 0.1)
]

index_row = filtered_row.index[0]

# Registro los resultados en MLFlow
with mlflow.start_run():

    # Almaceno los valores de los hiperparámetros
    for key, value in best_params.items():
        mlflow.log_param(key, value)

    # Registra las métricas de cada fold para cada métrica
    for metric in METRICS.keys():
        
        # Media
        mlflow.log_metric(f"mean_train_{metric}", df_results[f"mean_train_{metric}"][index_row])
        mlflow.log_metric(f"mean_test_{metric}", df_results[f"mean_test_{metric}"][index_row])

        # Desviación típica
        mlflow.log_metric(f"std_train_{metric}", df_results[f"std_train_{metric}"][index_row])
        mlflow.log_metric(f"std_test_{metric}", df_results[f"std_test_{metric}"][index_row])

        for i in range(5):

            # Resultados de entrenamiento en cada fold
            mlflow.log_metric(f"train_{metric}fold{i}", df_results[f"split{i}_train_{metric}"][index_row])
            # Resultados de validación en cada fold
            mlflow.log_metric(f"test_{metric}fold{i}", df_results[f"split{i}_test_{metric}"][index_row])

    # Establece una etiqueta que describe el propósito de esta ejecución
    mlflow.set_tag("Información", "GridSearch 144 combinaciones")

    # Infiere el signature del modelo, que describe el tipo de entrada y salida del modelo
    signature = infer_signature(X_scaled_train, grid_search.best_estimator_.predict(X_scaled_train))

    # Registra el modelo
    model_info = mlflow.sklearn.log_model(
        sk_model=grid_search,
        artifact_path="mlp_model",
        signature=signature,
        input_example=X_scaled_train,
        registered_model_name="mlp_gridsearch",
    )

2024/04/16 15:32:01 WARNING mlflow.utils.validation: Tag value '[{"run_id": "20732a375f774642b88efb425eccba0a", "artifact_path": "mlp_model", "utc_time_created": "2...' (24353 characters) is truncated to 5000 characters to meet the length limit.
Registered model 'mlp_gridsearch' already exists. Creating a new version of this model...
Created version '3' of model 'mlp_gridsearch'.


### 3.2 Random Search
[RandomizedSearchCV - sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)

In [126]:
# Defino las distribuciones de los parámetros
param_grid = {
    'classifier__hidden_layer_sizes': [(50, 50), (100, 100), (150, 150), (200, 100)],
    'classifier__activation': ['logistic'],
    'classifier__early_stopping': [True],
    'classifier__alpha': [0.0001, 0.001, 0.01, 0.1],
    'classifier__learning_rate': ['constant', 'invscaling', 'adaptive'],
    'classifier__learning_rate_init': [0.001, 0.01, 0.1]
}

# Inicializo RandomizedSearch
N_ITER = 336
random_search = RandomizedSearchCV(estimator=pipeline, param_distributions=param_dist, n_iter=N_ITER, cv=kf,
                                    scoring=METRICS, return_train_score=True,
                                    refit="balanced_accuracy",
                                    verbose=1, n_jobs=-1)

random_search.fit(X_scaled_train, y_scaled_train)

# Resultados
cv_results_rand = random_search.cv_results_
best_params_rand = random_search.best_params_

Fitting 5 folds for each of 144 candidates, totalling 720 fits


In [134]:
best_params_rand

{'classifier__activation': 'logistic',
 'classifier__alpha': 0.09884255446085931,
 'classifier__early_stopping': True,
 'classifier__hidden_layer_sizes': (200, 100),
 'classifier__learning_rate': 'adaptive',
 'classifier__learning_rate_init': 0.09338444290590295}

In [136]:
# Convertimos los resultados de la validación cruzada en un dataframe
df_results = pd.DataFrame(random_search.cv_results_)
df_results

# Filtrando la fila con condiciones específicas
filtered_row = df_results.loc[
    (df_results['param_classifier__activation'] == 'logistic') &
    (df_results['param_classifier__alpha'] == 0.09884255446085931) &
    (df_results['param_classifier__early_stopping'] == True) &
    (df_results['param_classifier__hidden_layer_sizes'] == (200, 100)) &
    (df_results['param_classifier__learning_rate'] == 'adaptive') &
    (df_results['param_classifier__learning_rate_init'] == 0.09338444290590295)
]

index_row = filtered_row.index[0]

# Registro los resultados en MLFlow
with mlflow.start_run():

    # Almaceno los valores de los hiperparámetros
    for key, value in best_params.items():
        mlflow.log_param(key, value)

    # Registra las métricas de cada fold para cada métrica
    for metric in METRICS.keys():
        
        # Media
        mlflow.log_metric(f"mean_train_{metric}", df_results[f"mean_train_{metric}"][index_row])
        mlflow.log_metric(f"mean_test_{metric}", df_results[f"mean_test_{metric}"][index_row])

        # Desviación típica
        mlflow.log_metric(f"std_train_{metric}", df_results[f"std_train_{metric}"][index_row])
        mlflow.log_metric(f"std_test_{metric}", df_results[f"std_test_{metric}"][index_row])

        for i in range(5):

            # Resultados de entrenamiento en cada fold
            mlflow.log_metric(f"train_{metric}fold{i}", df_results[f"split{i}_train_{metric}"][index_row])
            # Resultados de validación en cada fold
            mlflow.log_metric(f"test_{metric}fold{i}", df_results[f"split{i}_test_{metric}"][index_row])

    # Establece una etiqueta que describe el propósito de esta ejecución
    mlflow.set_tag("Información", "RandomizedSearch 144 combinaciones")

    # Infiere el signature del modelo, que describe el tipo de entrada y salida del modelo
    signature = infer_signature(X_scaled_train, random_search.best_estimator_.predict(X_scaled_train))

    # Registra el modelo
    model_info = mlflow.sklearn.log_model(
        sk_model=random_search,
        artifact_path="mlp_model",
        signature=signature,
        input_example=X_scaled_train,
        registered_model_name="mlp_randomizedsearch",
    )

2024/04/16 15:42:06 WARNING mlflow.utils.validation: Tag value '[{"run_id": "a8589311804e480cb08df355d3fd4312", "artifact_path": "mlp_model", "utc_time_created": "2...' (24353 characters) is truncated to 5000 characters to meet the length limit.
Registered model 'mlp_randomizedsearch' already exists. Creating a new version of this model...
Created version '4' of model 'mlp_randomizedsearch'.
